# Exercise 4

In [1]:
import sqlite3

In [2]:
# Connect to the SQLite database
conn = sqlite3.connect('database/db.sqlite')
c = conn.cursor()

In [3]:
# Load transactions from database
c.execute("SELECT * FROM transactions")
transactions = c.fetchall()
print(transactions)

[(1, 1, 2, 150.0, '2023-11-13 21:36:58.939091')]


In [4]:
# Update amount
c.execute("UPDATE transactions SET amount = 100 WHERE id = 1")

In [5]:
# Load transactions from database
c.execute("SELECT * FROM transactions")
transactions = c.fetchall()
print(transactions)

[(1, 1, 2, 100.0, '2023-11-13 21:36:58.939091')]
